In [1]:
import os

import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

tqdm.pandas()

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
df = None
path = "/kaggle/input/preprocesseddataset/merged_prs"
for file in tqdm(os.listdir(path)):
    if not file.endswith(".parquet"):
        continue
    if df is None:
        df = pd.read_parquet(f"{path}/{file}")
    else:
        tmp_df = pd.read_parquet(f"{path}/{file}")
        tmp_df["diff_len"] = tmp_df["diff"].str.len()
        tmp_df = tmp_df[tmp_df["diff_len"] < 100_000]
        df = pd.concat([df, tmp_df.sample(min(1_000, tmp_df.shape[0]))], ignore_index=True)
df = df.reset_index(drop=True)
df["body"] = df["body"].fillna("")
df = df[df["diff"].str.len() > 0]
df.head()

100%|██████████| 86/86 [01:16<00:00,  1.12it/s]


,title,diff,body,url,created_at,closed_at,merged_at,updated_at,diff_len
0,Add acme library usage example (http-01),diff --git a/acme/examples/http01_example.py b...,Here is a usage example of the ACME API explor...,https://api.github.com/repos/certbot/certbot/p...,2018-01-26T08:48:31Z,2019-02-23T02:02:43Z,2019-02-23T02:02:43Z,2022-09-10T18:42:56Z,NaN
1,Land release changes,diff --git a/acme/setup.py b/acme/setup.py\nin...,This PR should be merged and not squashed to k...,https://api.github.com/repos/certbot/certbot/p...,2018-01-25T21:51:18Z,2018-01-25T22:20:44Z,2018-01-25T22:20:44Z,2018-01-25T22:21:13Z,NaN
2,Update certbot-auto and help,diff --git a/certbot-auto b/certbot-auto\ninde...,,https://api.github.com/repos/certbot/certbot/p...,2018-01-25T21:50:42Z,2018-01-25T23:29:39Z,2018-01-25T23:29:39Z,2018-01-25T23:29:42Z,NaN
3,Fix --no-bootstrap on CentOS/RHEL 6,diff --git a/letsencrypt-auto-source/letsencry...,Cherry picked from #5476,https://api.github.com/repos/certbot/certbot/p...,2018-01-25T06:25:42Z,2018-01-25T06:59:20Z,2018-01-25T06:59:20Z,2018-01-25T06:59:25Z,NaN
4,Fix Nginx redirect issue,diff --git a/certbot-nginx/certbot_nginx/confi...,Cherry picks code from #5479.,https://api.github.com/repos/certbot/certbot/p...,2018-01-25T06:24:07Z,2018-01-25T06:49:52Z,2018-01-25T06:49:52Z,2018-01-25T06:49:55Z,NaN


In [4]:
df["repo_name"] = df["url"].apply(lambda x: x.split("/")[4] + "/" + x.split("/")[5])

In [5]:
diff = ""


def count_tokens(row):
    diff = row["diff"]
    return len(tokenizer.encode(diff))

In [6]:
df["diff_len"] = df.sample(30_000).progress_apply(count_tokens, axis=1)

100%|██████████| 30000/30000 [02:57<00:00, 169.26it/s]


In [7]:
df = df[(df["diff_len"] < 4_000) & (df["diff_len"] > 100)]

In [8]:
df["diff_len"].describe()

count    26100.000000
mean       817.421456
std        850.599546
min        101.000000
25%        229.000000
50%        458.000000
75%       1067.000000
max       3999.000000
Name: diff_len, dtype: float64

In [9]:
df["repo_name"].value_counts().head(10)

repo_name
certbot/certbot              1723
psf/requests                  568
django/django                 553
ytdl-org/youtube-dl           545
python/cpython                543
commaai/openpilot             536
ultralytics/yolov5            536
scikit-learn/scikit-learn     529
keras-team/keras              523
psf/black                     519
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df = df[df["repo_name"].isin((df["repo_name"].value_counts() > 10).index.tolist())]

In [12]:
train, other = train_test_split(df, test_size=2000, train_size=4000, stratify=df["repo_name"])
test, val = train_test_split(other, test_size=0.5, train_size=0.5)

In [13]:
train.shape, test.shape, val.shape

((4000, 10), (1000, 10), (1000, 10))

In [14]:
train.to_parquet("sampled.parquet")
test.to_parquet("test.parquet")
val.to_parquet("val.parquet")